In [4]:
import cvxpy as cvx
import numpy as np

### Setup Constants

In [5]:
l = 0.98
g = 9.8
m = 0.825
M = 8.085

tol = 1e-8
i = 0
MAX_ITER = 25

$\alpha$ for bisection

In [6]:
alpha_up = 800
alpha_lo = 0

### Dynamics

In [7]:
A = np.array([[0, 1, 0, 0], [0, 0, -(m*g/M), 0], [0, 0, 0, 1], [0, 0, g/l, 0]])
B = np.array([[0], 
              [1/M],
              [0],
              [-1/(M*l)]])
I = np.eye(4)

In [8]:
Q = cvx.Semidef(4)
Z = cvx.Variable(1,4)
obj = cvx.Minimize(0)

### Change of Variables for the Lyapunov Function
A change of variables is required to allow for a $K$ to be found.
$$ A_{cl}^TP + PA_{cl} + 2\alpha P < 0 $$
$$ (A + BK)^TP + P(A + BK) + 2\alpha P < 0 $$
$$ A^TP + K^TB^TP + PA + PBK + 2\alpha P < 0 $$

However, it is impossible to find a K from the above equation and so pre- and post-multiply by $P^{-1}$. 
$$ P^{-1}A^T + P^{-1}K^TB^T + AP^{-1} + BKP^{-1} + 2\alpha P^{-1} < 0 $$

And set $Q = P^{-1}$ and $Z = KP^{-1}$. Which becomes:
$$ QA^T + Z^TB^T + AQ + BZ + 2\alpha Q < 0 $$

In [ ]:
while(abs(alpha_up - alpha_lo) > tol and i < MAX_ITER):
    i += 1
    alpha = 0.5*(alpha_up + alpha_lo)
    const = [Q*A.T + Z.T*B.T + A*Q + B*Z + 2*alpha*Q < -I, Q > I]
    
    prob = cvx.Problem(obj, const)
    prob.solve()
    
    if prob.status == "infeasible":
        alpha_up = alpha
    else:
        alpha_lo = alpha

In [9]:
alpha = 0.
const = [Q*A.T + Z.T*B.T + A*Q + B*Z + 2*alpha*Q < -I, Q > I]

In [10]:
prob = cvx.Problem(obj, const)
prob.solve(verbose=True)

     pcost       dcost       gap    pres   dres   k/t
 0:  0.0000e+00  8.0000e+00  7e+01  4e+00  3e+01  1e+00
 1:  0.0000e+00  4.2479e+02  2e+05  1e+02  1e+03  2e+02
 2:  0.0000e+00  1.0011e+02  2e+03  6e+00  5e+01  9e+01
 3:  0.0000e+00  2.5309e+03  5e+04  6e+00  5e+01  3e+03
 4:  0.0000e+00  3.6109e+04  9e+05  7e+00  6e+01  4e+04
 5:  0.0000e+00  1.4799e+06  5e+07  1e+01  8e+01  1e+06
 6:  0.0000e+00  5.9369e+07  3e+09  1e+01  1e+02  6e+07
 7:  0.0000e+00  4.5049e+09  2e+11  1e+01  1e+02  5e+09
Certificate of primal infeasibility found.


inf